In [2]:

# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# # TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote, urlparse
# from urllib.error import HTTPError
# from zipfile import ZipFile
# import tarfile
# import shutil

# CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'home-credit-credit-risk-model-stability:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F50160%2F7602123%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T192212Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4e5dd0fd326c400fe8ccc714a10bef6da44c1517fa65467d6bc60ae9bb9df913c1dceff732cb1b12f0d4cf5b7c32f3c20bf299f52b8b237332f8a795d480e573f354448e605bd754bb643d076696e8a7a80d0304057c6f790eb0312241535c333dcc5251549c110f32cc1c13709ef4f2a4937850ad3d850eb617928c96376eca52d94545e99cbc01130648d04b8bfff0f4537e8ffc3650afe31360b30762ab7619e417ada27e095f7abe73e4d5866d7f5b88ecb805adad5f2c48bafc6f2067ba722a8e3bd87bba8c75ce3e90e8f3ccc69110a679b1cd0e10832e9dc3538a83cbf9824320a50aa2e2a43b50949ef721e48dbc6ab3382d844c0e10d3f1f88e9626'

# KAGGLE_INPUT_PATH='/kaggle/input'
# KAGGLE_WORKING_PATH='/kaggle/working'
# KAGGLE_SYMLINK='kaggle'

# !umount /kaggle/input/ 2> /dev/null
# shutil.rmtree('/kaggle/input', ignore_errors=True)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

# try:
#   os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
# except FileExistsError:
#   pass
# try:
#   os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
# except FileExistsError:
#   pass

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('Data source import complete.')


In [3]:
# !pip install scikit-learn numpy pandas torch polars scipy


import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import polars.selectors as cs
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as mn

dataPath = "/home/brett/Desktop/tutorials/kag/ds/"

In [4]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
        if col[-1] in ("D"):
            df = df.with_columns(pl.col(col).cast(pl.Date).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

# Load feature_def

In [5]:
def get_feature_definitions(columns):
    return pl.DataFrame({'Variable': columns}).join(
        feature_def,
        on = 'Variable',
        how = 'left',
    )

feature_def = pl.read_csv(dataPath + "feature_definitions.csv")

# Load Basetable

In [6]:
# train
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
train_basetable = train_basetable.with_columns(pl.col('date_decision').str.to_date())

# test
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
test_basetable = test_basetable.with_columns(pl.col('date_decision').str.to_date())

# Check Basetable

In [7]:
# print(train_basetable.columns)
# print('\n')
# print(train_basetable.shape)
# print('\n')
# display(train_basetable.head())

# Load depth=0

In [8]:
# train
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)

# test
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)

# Check depth=0

In [9]:
# print(train_static.columns)
# print('\n')
# print(train_static.shape)
# print('\n')
# display(train_static.head())

In [10]:
# print(train_static_cb.columns)
# print('\n')
# print(train_static_cb.shape)
# print('\n')
# display(train_static_cb.head())

# depth=1에 해당하는 테이블

Internal file
> `applprev_1` `debitcard_1` `deposit_1` `other_1` `person_1`

External file
> `credit_bureau_b_1`

In [11]:
## applprev_1
# train
train_applprev_1 = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/train/train_applprev_1_1.csv').pipe(set_table_dtypes),
    ],
    how='vertical_relaxed',
)

# test
test_applprev_1 = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/test/test_applprev_1_1.csv').pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/test/test_applprev_1_2.csv').pipe(set_table_dtypes),
    ],
    how='vertical_relaxed',
)

In [12]:
# train
train_debitcard_1 = pl.read_csv(dataPath + "csv_files/train/train_debitcard_1.csv").pipe(set_table_dtypes)
train_deposit_1 = pl.read_csv(dataPath + "csv_files/train/train_deposit_1.csv").pipe(set_table_dtypes)
train_other_1 = pl.read_csv(dataPath + 'csv_files/train/train_other_1.csv').pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes)
train_credit_bureau_b_1 = pl.read_csv(dataPath + 'csv_files/train/train_credit_bureau_b_1.csv').pipe(set_table_dtypes)
train_tax_a = pl.read_csv(dataPath + "csv_files/train/train_tax_registry_a_1.csv").pipe(set_table_dtypes)

# test
test_debitcard_1 = pl.read_csv(dataPath + "csv_files/test/test_debitcard_1.csv").pipe(set_table_dtypes)
test_deposit_1 = pl.read_csv(dataPath + "csv_files/test/test_deposit_1.csv").pipe(set_table_dtypes)
test_other_1 = pl.read_csv(dataPath + 'csv_files/test/test_other_1.csv').pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes)
test_credit_bureau_b_1 = pl.read_csv(dataPath + 'csv_files/test/test_credit_bureau_b_1.csv').pipe(set_table_dtypes)
test_tax_a = pl.read_csv(dataPath + "csv_files/test/test_tax_registry_a_1.csv").pipe(set_table_dtypes)

In [13]:
print(
    train_applprev_1.shape,
    train_debitcard_1.shape,
    train_deposit_1.shape,
    train_other_1.shape,
    train_person_1.shape,
    train_credit_bureau_b_1.shape,
    train_tax_a.shape,
)

(6525979, 41) (157302, 6) (145086, 5) (51109, 7) (2973991, 37) (85791, 45) (3275770, 5)


# Feature Engineering

In [14]:
## applprev_1
app_cols = [
    'case_id',
    'maxdpdtolerance_577P',
    'employedfrom_700D',
    'pmtnum_8L',
    'creationdate_885D',
    'credamount_590A',
    'mainoccupationinc_437A',
    'firstnonzeroinstldate_307D',
    'dtlastpmtallstes_3545839D',
    'annuity_853A',
    'familystate_726L',
    'approvaldate_319D',
    'district_544M',
    'dateactivated_425D',
    'education_1138M',
]

# train
train_applprev_1 = train_applprev_1[app_cols]
train_applprev_1 = train_applprev_1.unique(subset='case_id', keep='first')

# test
test_applprev_1 = test_applprev_1[app_cols]
test_applprev_1 = test_applprev_1.unique(subset='case_id', keep='first')

In [15]:
## case_id를 기준으로 그룹화한 후, aggregation functions

# applprev_1
train_applprev_1_feats_1 = train_applprev_1.group_by("case_id").agg(
    pl.col("pmtnum_8L").max().alias("max_pmtnum_8L"))

# debitcard_1
train_debitcard_1_feats = train_debitcard_1.group_by("case_id").agg(
    # 체크카드의 평균 잔액의 최대값
    pl.col('last180dayaveragebalance_704A').max().alias('max_last180dayaveragebalance_704A'),
    # 180일간의 체크카드 거래액의 최대값
    pl.col('last180dayturnover_1134A').max().alias('max_last180dayturnover_1134A'),
    # 30일간의 체크카드 거래액의 최대값
    pl.col("last30dayturnover_651A").max().alias("max_last30dayturnover_651A"),
).sort(by='case_id')

# deposit_1
train_deposit_1_feats = train_deposit_1.group_by('case_id').agg(
    # 예/적금의 합계
    pl.sum('amount_416A').alias('sum_amount_416A'),
).sort(by='case_id')

# other_1
train_other_1_feats = train_other_1.group_by("case_id").agg(
    # 입금액의 합계
    pl.col("amtdepositincoming_4809444A").sum().alias("sum_amtdepositincoming_4809444A"),
    # 출금액의 합계
    pl.col("amtdepositoutgoing_4809442A").sum().alias("sum_amtdepositoutgoing_4809442A"),
).sort(by='case_id')

# person_1
train_person_1_feats_1 = train_person_1.select(['case_id', 'incometype_1044T','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

train_person_1_feats_2 = train_person_1.select(['case_id', 'language1_981M','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

# credit_bureau_b_1
train_credit_bureau_b_1_feats = train_credit_bureau_b_1.group_by('case_id').agg(
    pl.col('installmentamount_644A').mean().alias('mean_installmentamount_644A'),
    pl.col('maxdebtpduevalodued_3940955A').mean().alias('mean_maxdebtpduevalodued_3940955A'),
    pl.col('overdueamountmax_950A').mean().alias('mean_overdueamountmax_950A'),
    pl.col('residualamount_1093A').mean().alias('mean_residualamount_1093A'),
).sort(by='case_id')


## ⭐키워드 6가지 중 어떤걸 선택할지?
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "D", "M", 'P', 'T', "L"):
        selected_static_cols.append(col)
print(selected_static_cols)

# static_cb_0에서는 D를 사용할 필요가 없어보임.
selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M", 'P', 'T', 'L'):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)


## Join
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
# ).join(
#     train_applprev_1, how="left", on="case_id"
).join(
    train_debitcard_1_feats, how="left", on="case_id"
).join(
    train_deposit_1_feats, how="left", on="case_id"
).join(
    train_other_1_feats, how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_1_feats, how="left", on="case_id"
).join(
    train_applprev_1_feats_1, how="left", on="case_id"
# ).join(
#     train_tax_a, how="left", on="case_id"
)


## num_group1이 0인 경우 대출을 신청한 사람
# train_debitcard_1_feats_2 = train_debitcard_1.select(["case_id", "num_group1"]).filter(
#     pl.col("num_group1") == 0 # 필터링 후
# ).drop("num_group1") # 컬럼 삭제

['actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'bankacctype_710L', 'cardtype_51L', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'currdebt_22A',

In [16]:
def calculate_date_difference(df):
    for col in df.columns:
        if col[-1] == "D":
            df = df.with_columns(
                (pl.col('date_decision') - pl.col(col)).cast(pl.Int64).alias(col)
            )
    return df

In [17]:
data = calculate_date_difference(data)

In [18]:
test_applprev_1_feats_1 = test_applprev_1.group_by("case_id").agg(
    pl.col("pmtnum_8L").max().alias("max_pmtnum_8L"))

test_debitcard_1_feats = test_debitcard_1.group_by("case_id").agg(
    pl.col('last180dayaveragebalance_704A').max().alias('max_last180dayaveragebalance_704A'),
    pl.col('last180dayturnover_1134A').max().alias('max_last180dayturnover_1134A'),
    pl.col("last30dayturnover_651A").max().alias("max_last30dayturnover_651A"),
).sort(by='case_id')

test_deposit_1_feats = test_deposit_1.group_by('case_id').agg(
    pl.sum('amount_416A').alias('sum_amount_416A')
).sort(by='case_id')

test_other_1_feats = test_other_1.group_by("case_id").agg(
    pl.col("amtdepositincoming_4809444A").sum().alias("sum_amtdepositincoming_4809444A"),
    pl.col("amtdepositoutgoing_4809442A").sum().alias("sum_amtdepositoutgoing_4809442A"),
).sort(by='case_id')

test_person_1_feats_1 = test_person_1.select(['case_id', 'incometype_1044T','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

test_person_1_feats_2 = test_person_1.select(['case_id', 'language1_981M','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

test_credit_bureau_b_1_feats = test_credit_bureau_b_1.group_by('case_id').agg(
    pl.col('installmentamount_644A').mean().alias('mean_installmentamount_644A'),
    pl.col('maxdebtpduevalodued_3940955A').mean().alias('mean_maxdebtpduevalodued_3940955A'),
    pl.col('overdueamountmax_950A').mean().alias('mean_overdueamountmax_950A'),
    pl.col('residualamount_1093A').mean().alias('mean_residualamount_1093A'),
).sort(by='case_id')


data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
# ).join(
#     test_applprev_1, how="left", on="case_id"
).join(
    test_debitcard_1_feats, how="left", on="case_id"
).join(
    test_deposit_1_feats, how="left", on="case_id"
).join(
    test_other_1_feats, how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_1_feats, how="left", on="case_id"
).join(
    test_applprev_1_feats_1, how="left", on="case_id"
# ).join(
#     test_tax_a, how="left", on="case_id"
)

In [19]:
data_submission = calculate_date_difference(data_submission)

In [24]:
# !conda install pyarrow -y 

case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

print('\n')
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")




# # !conda install pyarrow -y 
# case_ids = data["case_id"].unique().shuffle(seed=1)
# case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
# case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

# cols_pred = []
# for col in data.columns:
#     if col[-1].isupper() and col[:-1].islower():
#         cols_pred.append(col)

# print(cols_pred)

# def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
#     return (
#         data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
#         data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
#         data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
#     )

# base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
# base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
# base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

# for df in [X_train, X_valid, X_test]:
#     df = convert_strings(df)

# !conda install pyarrow -y 


from sklearn.preprocessing import OneHotEncoder
import numpy as np
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import torch 
import polars as pl
import numpy as np
from scipy.stats import norm


case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if (col[-1].isupper() and col[:-1].islower()) or col == 'date_decision' or col == 'totaldebt_9A':
        cols_pred.append(col)

# print(cols_pred)

# Add this line to print all column names from the dataset
print("All column names in the dataset:", cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

# base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
# base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
# base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

mx_smpls = 900000
# Convert from Polars to Pandas and then take only the first 100 samples for training
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
print(base_train.shape)
base_train, X_train, y_train = base_train.head(mx_smpls), X_train.head(mx_smpls), y_train.head(mx_smpls)

# Convert from Polars to Pandas and then take only the first 100 samples for validation
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_valid, X_valid, y_valid = base_valid.head(mx_smpls), X_valid.head(mx_smpls), y_valid.head(mx_smpls)

# Convert from Polars to Pandas and then take only the first 100 samples for testing
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)
base_test, X_test, y_test = base_test.head(mx_smpls), X_test.head(mx_smpls), y_test.head(mx_smpls)


for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)


covid_start_date = pd.to_datetime('2020-01-01')

def convert_date_to_binary(df):
    # Ensure date_decision is in datetime format
    df['date_decision'] = pd.to_datetime(df['date_decision'])
    # Convert date_decision to binary: 1 if during or after COVID start, 0 otherwise
    df['date_decision'] = (df['date_decision'] >= covid_start_date).astype(int)
    return df

# Now apply this function to your DataFrames
X_train = convert_date_to_binary(X_train)
X_valid = convert_date_to_binary(X_valid)
X_test = convert_date_to_binary(X_test)

    
print("Sample values from X_train:")
for col in X_train.columns:
    print(f"{col}: {X_train[col].sample(10).to_list()}")

##### cutoff 

# Function to manually create one-hot encoding for numeric data
def one_hot_numeric(series, num_bins=100):
    clean_series = series.dropna()
    mu, sigma = clean_series.mean(), clean_series.std()
    percentiles = series.apply(lambda x: norm.cdf(x, mu, sigma) if pd.notnull(x) else np.nan)
    bins = np.linspace(0, 1, num_bins + 1)
    one_hot_matrix = np.zeros((len(series), num_bins))
    for i, val in enumerate(percentiles):
        if pd.notnull(val):
            bin_index = np.digitize(val, bins) - 1
            one_hot_matrix[i, min(bin_index, num_bins - 1)] = 1
        else:
            one_hot_matrix[i, -1] = 1
    return one_hot_matrix

# Function to process categorical data
def process_categorical(series):
    return pd.get_dummies(series, dummy_na=True, prefix='category')

# Process each column in data_df

#### memory error for the below 

# def preprocess(data_df):
#     one_hot_encoded_tensors = []
#     for column in data_df:
#         series = data_df[column]
#         if series.dtype == 'float64':
#             one_hot_matrix = one_hot_numeric(series)
#             one_hot_encoded_tensors.append(torch.tensor(one_hot_matrix, dtype=torch.float32))
#         elif series.dtype == 'object':
#             one_hot_df = process_categorical(series)
#             one_hot_encoded_tensors.append(torch.tensor(one_hot_df.values, dtype=torch.float32))

#     def convert_one_hot_to_tokens(one_hot_tensors):
#         token_sequences = []
#         offset = 0
#         for tensor in one_hot_tensors:
#             tokens = [(torch.argmax(vector).item() + offset) if vector.sum() != 0 else -1 for vector in tensor]
#             token_sequences.append(tokens)
#             offset += tensor.shape[1]
#         return token_sequences

#     def combine_tokens(token_sequences):
#         combined_sequences = []
#         for sequence in zip(*token_sequences):
#             combined_sequences.append(list(sequence))
#         return combined_sequences

#     token_sequences = convert_one_hot_to_tokens(one_hot_encoded_tensors)
#     combined_sequences = combine_tokens(token_sequences)
#     return combined_sequences


from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Function to manually create one-hot encoding for numeric data
def one_hot_numeric(series, num_bins=10):
    clean_series = series.dropna()
    mu, sigma = clean_series.mean(), clean_series.std()
    percentiles = series.apply(lambda x: norm.cdf(x, mu, sigma) if pd.notnull(x) else np.nan)
    bins = np.linspace(0, 1, num_bins + 1)
    one_hot_matrix = np.zeros((len(series), num_bins))
    for i, val in enumerate(percentiles):
        if pd.notnull(val):
            bin_index = np.digitize(val, bins) - 1
            one_hot_matrix[i, min(bin_index, num_bins - 1)] = 1
        else:
            one_hot_matrix[i, -1] = 1
    return one_hot_matrix

# Function to process categorical data
def process_categorical(series):
    # Instead of using pd.get_dummies, use OneHotEncoder for more efficiency
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    one_hot_array = encoder.fit_transform(series.values.reshape(-1, 1))
    return one_hot_array

# Process each column in data_df
def preprocess(data_df):
    one_hot_encoded_arrays = []
    for column in data_df:
        series = data_df[column]
        if series.dtype == 'float64':
            one_hot_matrix = one_hot_numeric(series)
            one_hot_encoded_arrays.append(one_hot_matrix)
        elif series.dtype == 'object':
            one_hot_array = process_categorical(series)
            one_hot_encoded_arrays.append(one_hot_array)

    # Combine the one-hot encoded arrays
    combined_sequences = np.hstack(one_hot_encoded_arrays)
    return combined_sequences


# tr_pre = X_train
# val_pre = X_valid
# test_pre = X_test

# print(tr_pre.shape, val_pre.shape)  


combined_sequences = preprocess(data_df=X_train)
combined_sequences_val = preprocess(data_df=X_train)

# print("Combined token sequences for input to transformer:")
# for seq in combined_sequences:
#     print(seq)

['actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'bankacctype_710L', 'cardtype_51L', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'currdebt_22A',

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torch.optim import Adam
# import torch

# # Clear cache memory
# torch.cuda.empty_cache()

# # Reset peak memory statistics
# torch.cuda.reset_peak_memory_stats()

# # If you want to clear the memory for a specific device, you can specify the device index
# device_index = 0  # For example, for the first CUDA device
# torch.cuda.empty_cache(device_index)
# torch.cuda.reset_peak_memory_stats(device_index)

# import pickle

# # Assuming combined_sequences and combined_sequences_val are lists or similar structures
# def save_data_to_pickle(data, filename):
#     with open(filename, 'wb') as file:
#         pickle.dump(data, file)


# save_data_to_pickle(combined_sequences, 'combined_sequences.pkl')

# # Save combined_sequences_val
# save_data_to_pickle(combined_sequences_val, 'combined_sequences_val.pkl')


# Training LightGBM

Minimal example of LightGBM training is shown below.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch

import pickle

def load_data_from_pickle(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data

# Load combined_sequences and combined_sequences_val
combined_sequences = load_data_from_pickle('combined_sequences_lg.pkl')
combined_sequences_val = load_data_from_pickle('combined_sequences_val_lg.pkl')


# import torch
import math



# print(f"Train: {X_train.shape}")
# print(f"Valid: {X_valid.shape}")
# print(f"Test: {X_test.shape}")

from sklearn.preprocessing import MinMaxScaler
from torch import nn, optim




import torch
import torch.nn as nn
import math

# class SimpleTransformer(nn.Module):
#     def __init__(self, input_dim, model_dim, num_classes, num_heads=2, num_layers=1, dropout=0.1, mlp_layers=3):
#         super(SimpleTransformer, self).__init__()
#         self.embed = nn.Embedding(input_dim, model_dim)
#         encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
#         self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
#         # MLP layers
#         mlp_modules = [nn.Linear(model_dim, model_dim * 2), nn.ReLU()]
#         for _ in range(mlp_layers - 1):
#             mlp_modules.extend([nn.Linear(model_dim * 2, model_dim * 2), nn.ReLU()])
#         mlp_modules.append(nn.Linear(model_dim * 2, num_classes))
#         self.mlp = nn.Sequential(*mlp_modules)
        
#         # Positional encoding
#         self.positional_encoding = self.create_positional_encoding(175, model_dim)  # Keep max_len at 174

#     def create_positional_encoding(self, max_len, model_dim):
#         pe = torch.zeros(max_len, model_dim)
#         position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
#         div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)  # Add batch dimension for broadcasting
#         return pe.cuda()

#     def forward(self, x):
#         x = self.embed(x)  # Convert token indices to embeddings
#         x += self.positional_encoding  # Add positional encoding to embeddings
#         x = self.transformer_encoder(x)
#         x = torch.mean(x, dim=1)  # Average pooling across the sequence dimension
#         x = self.mlp(x)  # Pass through MLP
#         return x
# # Define a simple Transformer model for demonstration purposes
    

class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_classes, num_heads=2, num_layers=1, dropout=0.1):
        super(SimpleTransformer, self).__init__()
        self.embed = nn.Embedding(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(model_dim, num_classes)
        self.positional_encoding = self.create_positional_encoding(174+1, model_dim)  # Create positional encoding

    def create_positional_encoding(self, max_len, model_dim):
        """Create positional encoding matrix."""
        # Create a matrix that contains the positional encodings for all possible positions
        pe = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Add batch dimension for broadcasting
        return pe.cuda()


    def forward(self, x):
        x = self.embed(x)  # Convert token indices to embeddings
        x = x + self.positional_encoding# Add positional encoding to embeddings
        x = self.transformer_encoder(x)
        x = torch.mean(x, dim=1)  # Average pooling
        return self.output_layer(x)

# class TokenSequenceDataset(Dataset):
#     def __init__(self, sequences, labels):
#         self.sequences = sequences
#         self.labels = labels

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         sequence = torch.tensor(self.sequences[idx], dtype=torch.long)
#         label = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are long type for classification
#         return sequence, label

import pickle
import torch
from torch.utils.data import Dataset, DataLoader

# Function to load predictions from pickle files
def load_predictions(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# Function to convert prediction scores into integer tokens
def convert_scores_to_tokens(scores, num_bins=100):
    # Assuming scores are in the range [0, 1], scale them to [1, 100]
    return [int(score * (num_bins - 1)) + 10000 for score in scores]

# Your modified TokenSequenceDataset class
class TokenSequenceDataset(Dataset):
    def __init__(self, sequences, labels, prediction_scores):
        self.sequences = sequences
        self.labels = labels
        self.prediction_tokens = convert_scores_to_tokens(prediction_scores)
        assert len(self.sequences) == len(self.prediction_tokens), "Sequences and predictions must have the same length"

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Add the prediction token to the end of the sequence
        sequence_with_token = self.sequences[idx] + [self.prediction_tokens[idx]]
        sequence_tensor = torch.tensor(sequence_with_token, dtype=torch.long)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)  # Ensure labels are long type for classification
        return sequence_tensor, label_tensor

# Example usage
train_pred_scores = load_predictions('/home/brett/Desktop/tutorials/kag/y_pred_train.pkl')
valid_pred_scores = load_predictions('/home/brett/Desktop/tutorials/kag/y_pred_valid.pkl')



# # Create the dataset
# train_dataset = TokenSequenceDataset(train_sequences, train_labels, train_pred_scores)

# # Example of using DataLoader
# train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

train_dataset = TokenSequenceDataset(combined_sequences, y_train.to_numpy(), train_pred_scores)

val_dataset = TokenSequenceDataset(combined_sequences_val, y_valid.to_numpy(), valid_pred_scores)

# class TokenSequenceDataset(Dataset):
#     def __init__(self, sequences, labels):
#         self.sequences = sequences
#         self.labels = labels
#         # Define tokens for negative and positive classes
#         self.neg_token = 0000
#         self.pos_token = 1111

#     def __len__(self):
#         return len(self.sequences)

#     def __getitem__(self, idx):
#         # Copy the original sequence to avoid modifying the original data
#         sequence = list(self.sequences[idx])  # Convert to list if it's not already
#         label = self.labels[idx]
        
#         # Append special token based on the label
#         if label == 0:  # Assuming 0 is your negative class
#             sequence.append(self.neg_token)
#         else:  # Assuming 1 or any non-zero value is your positive class
#             sequence.append(self.pos_token)
        
#         # Convert to tensors
#         sequence_tensor = torch.tensor(sequence, dtype=torch.long)
#         label_tensor = torch.tensor(label, dtype=torch.long)  # Ensure labels are long type for classification
        
#         return sequence_tensor, label_tensor

# Model parameters
INPUT_DIM = max(max(seq) for seq in combined_sequences) + 1  # Plus one since we're using these as embeddings indices
MODEL_DIM = 128
NUM_CLASSES = 2  # Adjust based on your actual number of classes
NUM_HEADS = 16
NUM_LAYERS = 2
DROPOUT = 0.1

# # Initialize model, loss function, and optimizer
# model = SimpleTransformer(INPUT_DIM, MODEL_DIM, NUM_CLASSES, NUM_HEADS, NUM_LAYERS, DROPOUT).cuda
# loss_function = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters())

# train_dataset = TokenSequenceDataset(combined_sequences, y_train.to_numpy())  # Assuming y_train is your training labels
# val_dataset = TokenSequenceDataset(combined_sequences_val, y_valid.to_numpy())  # And y_valid for validation

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)  # Typically no shuffle for validation data


# from sklearn.metrics import roc_auc_score

# # Training loop adjustments
# num_epochs = 50  # Set this to your desired number of epochs
# for epoch in range(num_epochs):
#     print(epoch)
#     model.train()  # Set the model to training mode
#     for batch_sequences, batch_labels in train_dataloader:
#         optimizer.zero_grad()
#         outputs = model(batch_sequences)
#         loss = loss_function(outputs, batch_labels)
#         loss.backward()
#         optimizer.step()
    
#     # Print the average loss after each epoch (optional, you could add logic here to track and print this)

#     # Validation loop adjustments
#     model.eval()  # Set the model to evaluation mode
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():  # No need to track gradients for validation
#         for batch_sequences, batch_labels in val_dataloader:
#             outputs = model(batch_sequences)
#             all_labels.extend(batch_labels.tolist())  # Collecting actual labels

#             # Applying softmax to get probabilities, assuming your task is binary classification
#             # For multi-class, adjust accordingly
#             probas = outputs.softmax(dim=1)[:, 1]  # get the probability of the positive class
#             all_preds.extend(probas.tolist())

#     # Calculate AUC
#     # Ensure all_labels and all_preds are both populated with all data from validation set
#     auc_score = roc_auc_score(all_labels, all_preds)
#     print(f"Epoch {epoch}, Validation AUC: {auc_score}")

# # Training loop
# # Training loop
# for epoch in range(1):  # Increase number of epochs as needed
#     for i, (batch_sequences, batch_labels) in enumerate(dataloader):
#         optimizer.zero_grad()
#         outputs = model(batch_sequences)
#         loss = loss_function(outputs, batch_labels)  # Use the actual labels
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch {epoch}, Batch {i}, Loss: {loss.item()}")



# Initialize the MinMaxScaler
# scaler = MinMaxScaler()

# # Fit the scaler using only the training data and transform training data
# X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=[np.number]))  # Apply only to numeric columns
# X_train = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=[np.number]).columns)

# # Apply the same transformation to validation and test data
# X_valid_scaled = scaler.transform(X_valid.select_dtypes(include=[np.number]))
# X_valid = pd.DataFrame(X_valid_scaled, columns=X_valid.select_dtypes(include=[np.number]).columns)

# X_test_scaled = scaler.transform(X_test.select_dtypes(include=[np.number]))
# X_test = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=[np.number]).columns)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss function, and optimizer
model = SimpleTransformer(INPUT_DIM, MODEL_DIM, NUM_CLASSES, NUM_HEADS, NUM_LAYERS, DROPOUT).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Assuming y_train and y_valid are your training and validation labels respectively
# train_dataset = TokenSequenceDataset(combined_sequences, y_train.to_numpy())
# val_dataset = TokenSequenceDataset(combined_sequences_val, y_valid.to_numpy())


            
from torch.utils.data import WeightedRandomSampler

# Calculate class weights inversely proportional to their frequencies
class_sample_counts = [np.sum(y_train == t) for t in np.unique(y_train)]
class_weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
# Assign a weight to every data point based on its class
sample_weights = class_weights[y_train]

# Create a WeightedRandomSampler object
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=256, sampler=sampler)
val_dataloader = DataLoader(val_dataset, batch_size=256,  sampler=sampler)

# Training loop
# num_epochs = 50
# for epoch in range(num_epochs):
#     print(f'Epoch {epoch}/{num_epochs - 1}')
#     model.train()  # Set the model to training mode
#     for batch_sequences, batch_labels in train_dataloader:
#         batch_sequences, batch_labels = batch_sequences.to(device), batch_labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(batch_sequences)
#         loss = loss_function(outputs, batch_labels)
#         loss.backward()
#         optimizer.step()
    
#     # Validation loop
#     model.eval()  # Set the model to evaluation mode
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():  # No need to track gradients for validation
#         for batch_sequences, batch_labels in val_dataloader:
#             batch_sequences, batch_labels = batch_sequences.to(device), batch_labels.to(device)
#             outputs = model(batch_sequences)
#             all_labels.extend(batch_labels.tolist())  # Collecting actual labels

#             # Applying softmax to get probabilities, assuming your task is binary classification
#             probas = outputs.softmax(dim=1)[:, 1]  # get the probability of the positive class
#             all_preds.extend(probas.tolist())

#     # Calculate AUC
#     auc_score = roc_auc_score(all_labels, all_preds)
#     print(f"Validation AUC: {auc_score}")
        
    
    
num_epochs = 5000
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    model.train()  # Set the model to training mode
    total_train, correct_train = 0, 0
    for batch_sequences, batch_labels in train_dataloader:
        batch_sequences, batch_labels = batch_sequences.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model(batch_sequences)
        loss = loss_function(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += batch_labels.size(0)
        correct_train += (predicted == batch_labels).sum().item()
    
    train_accuracy = 100 * correct_train / total_train
    print(f'Training Accuracy: {train_accuracy}%')
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    total_val, correct_val = 0, 0
    with torch.no_grad():  # No need to track gradients for validation
        for batch_sequences, batch_labels in val_dataloader:
            batch_sequences, batch_labels = batch_sequences.to(device), batch_labels.to(device)
            outputs = model(batch_sequences)
            all_labels.extend(batch_labels.tolist())  # Collecting actual labels

            # Applying softmax to get probabilities, assuming your task is binary classification
            probas = outputs.softmax(dim=1)[:, 1]  # get the probability of the positive class
            all_preds.extend(probas.tolist())

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += batch_labels.size(0)
            correct_val += (predicted == batch_labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Accuracy: {val_accuracy}%')

    # Calculate AUC
    auc_score = roc_auc_score(all_labels, all_preds)
    print(f"Validation AUC: {auc_score}")


Epoch 0/4999
Training Accuracy: 50.397%
Validation Accuracy: 96.963%
Validation AUC: 0.6184197397314017
Epoch 1/4999
Training Accuracy: 64.928%
Validation Accuracy: 52.292%
Validation AUC: 0.6636793562133884
Epoch 2/4999
Training Accuracy: 68.023%
Validation Accuracy: 47.567%
Validation AUC: 0.6659052110279038
Epoch 3/4999
Training Accuracy: 68.604%
Validation Accuracy: 67.247%
Validation AUC: 0.6744656752910184
Epoch 4/4999
Training Accuracy: 69.073%
Validation Accuracy: 78.215%
Validation AUC: 0.6641615582066046
Epoch 5/4999
Training Accuracy: 68.586%
Validation Accuracy: 72.729%
Validation AUC: 0.66708478360396
Epoch 6/4999
Training Accuracy: 69.196%
Validation Accuracy: 78.851%
Validation AUC: 0.6387704317224152
Epoch 7/4999
Training Accuracy: 69.258%
Validation Accuracy: 82.985%
Validation AUC: 0.6640425676088684
Epoch 8/4999
Training Accuracy: 69.552%
Validation Accuracy: 83.083%
Validation AUC: 0.6588633115027827
Epoch 9/4999
Training Accuracy: 69.647%
Validation Accuracy: 76.87

KeyboardInterrupt: 

In [ ]:
# del model 
# # # Clear cache memory
# # torch.cuda.empty_cache()

# # # Reset peak memory statistics
# # torch.cuda.reset_peak_memory_stats()
# import torch
# from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import get_linear_schedule_with_warmup
# from sklearn.metrics import accuracy_score, roc_auc_score
# import numpy as np

# # Load the tokenizer and model from pre-trained BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

# class BertTokenSequenceDataset(Dataset):
#     def __init__(self, sequences, labels, tokenizer, max_length=512):
#         self.tokenizer = tokenizer
#         self.texts = sequences
#         self.labels = labels
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         inputs = self.tokenizer.encode_plus(
#             text,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             padding='max_length',
#             return_token_type_ids=True,
#             truncation=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'labels': torch.tensor(self.labels[idx], dtype=torch.long)
#         }

# # Assuming you have your sequences and labels prepared
# train_dataset = BertTokenSequenceDataset(combined_sequences, y_train, tokenizer)
# val_dataset = BertTokenSequenceDataset(combined_sequences_val, y_valid, tokenizer)

# train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# total_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# num_epochs = 3000  # You can adjust this
# for epoch in range(num_epochs):
#     model.train()
#     total_train_loss = 0
#     for batch in train_dataloader:
#         batch_sequences = batch['ids'].to(device)
#         batch_mask = batch['mask'].to(device)
#         batch_labels = batch['labels'].to(device)

#         model.zero_grad()
#         outputs = model(batch_sequences, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)
#         loss = outputs.loss
#         loss.backward()
#         total_train_loss += loss.item()
#         optimizer.step()
#         scheduler.step()

#     avg_train_loss = total_train_loss / len(train_dataloader)
#     print(f'Epoch {epoch}/{num_epochs - 1}, Training Loss: {avg_train_loss}')

#     # Validation loop
#     model.eval()
#     total_eval_accuracy = 0
#     total_eval_loss = 0
#     for batch in val_dataloader:
#         batch_sequences = batch['ids'].to(device)
#         batch_mask = batch['mask'].to(device)
#         batch_labels = batch['labels'].to(device)

#         with torch.no_grad():
#             outputs = model(batch_sequences, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)
            
#         loss = outputs.loss
#         total_eval_loss += loss.item()
#         logits = outputs.logits.detach().cpu().numpy()
#         label_ids = batch_labels.to('cpu').numpy()
#         predictions = np.argmax(logits, axis=1)
#         total_eval_accuracy += accuracy_score(label_ids, predictions)

#     avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
#     avg_val_loss = total_eval_loss / len(val_dataloader)
#     print(f'Epoch {epoch}/{num_epochs - 1}, Validation Accuracy: {avg_val_accuracy}, Validation Loss: {avg_val_loss}')


In [ ]:
# # del model 
# # # Clear cache memory
# # torch.cuda.empty_cache()

# # # Reset peak memory statistics
# # torch.cuda.reset_peak_memory_stats()


# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import get_linear_schedule_with_warmup
# from sklearn.metrics import accuracy_score, roc_auc_score
# import numpy as np

# # Assuming your device setup (CUDA or CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load the tokenizer and model from pre-trained BERT
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

# class BertTokenSequenceDataset(Dataset):
#     def __init__(self, sequences, labels, tokenizer, max_length=512):
#         self.tokenizer = tokenizer
#         self.texts = sequences
#         self.labels = labels
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         inputs = self.tokenizer.encode_plus(
#             text,
#             None,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             padding='max_length',
#             return_token_type_ids=True,
#             truncation=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']
#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long),
#             'labels': torch.tensor(self.labels[idx], dtype=torch.long)
#         }



# # Model parameters
# INPUT_DIM = max(max(seq) for seq in combined_sequences) + 1  # Plus one since we're using these as embeddings indices
# MODEL_DIM = 128
# NUM_CLASSES = 2  # Adjust based on your actual number of classes
# NUM_HEADS = 16
# NUM_LAYERS = 2
# DROPOUT = 0.1
# num_epochs = 30000
# # Assuming you have your sequences and labels prepared
# # combined_sequences, y_train, combined_sequences_val, y_valid need to be defined previously
# train_dataset = BertTokenSequenceDataset(combined_sequences, y_train, tokenizer)
# val_dataset = BertTokenSequenceDataset(combined_sequences_val, y_valid, tokenizer)


# from torch.utils.data import WeightedRandomSampler

# # Calculate class weights inversely proportional to their frequencies
# class_sample_counts = [np.sum(y_train == t) for t in np.unique(y_train)]
# class_weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
# # Assign a weight to every data point based on its class
# sample_weights = class_weights[y_train]

# # Create a WeightedRandomSampler object
# sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# train_dataloader = DataLoader(train_dataset, batch_size=12, sampler=sampler)
# val_dataloader = DataLoader(val_dataset, batch_size=12, sampler=sampler)

# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# total_steps = len(train_dataloader) * num_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# num_epochs = 3  # You can adjust this
# for epoch in range(num_epochs):
#     model.train()
#     total_train_loss = 0
#     for batch in train_dataloader:
#         batch_sequences = batch['ids'].to(device)
#         batch_mask = batch['mask'].to(device)
#         batch_labels = batch['labels'].to(device)

#         model.zero_grad()
#         outputs = model(batch_sequences, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)
#         loss = outputs.loss
#         loss.backward()
#         total_train_loss += loss.item()
#         optimizer.step()
#         scheduler.step()

#     avg_train_loss = total_train_loss / len(train_dataloader)
#     print(f'Epoch {epoch}/{num_epochs - 1}, Training Loss: {avg_train_loss}')

#     # Validation loop
#     model.eval()
#     total_eval_accuracy = 0
#     total_eval_loss = 0
#     all_labels = []
#     all_preds = []
#     for batch in val_dataloader:
#         batch_sequences = batch['ids'].to(device)
#         batch_mask = batch['mask'].to(device)
#         batch_labels = batch['labels'].to(device)

#         with torch.no_grad():
#             outputs = model(batch_sequences, token_type_ids=None, attention_mask=batch_mask, labels=batch_labels)

#         loss = outputs.loss
#         total_eval_loss += loss.item()
#         logits = outputs.logits.detach().cpu().numpy()
#         label_ids = batch_labels.to('cpu').numpy()
#         predictions = np.argmax(logits, axis=1)
#         total_eval_accuracy += accuracy_score(label_ids, predictions)

#         # Store all labels and predictions to calculate AUC later
#         all_labels.extend(label_ids)
#         all_preds.extend(logits[:, 1])  # Assuming the positive class is at index 1

#     avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
#     avg_val_loss = total_eval_loss / len(val_dataloader)
#     val_auc = roc_auc_score(all_labels, all_preds)  # Calculate AUC
#     print(f'Epoch {epoch}/{num_epochs - 1}, Validation Accuracy: {avg_val_accuracy}, Validation Loss: {avg_val_loss}, Validation AUC: {val_auc}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 4,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)


import numpy as np

# Generate predictions
y_pred_train = np.round(gbm.predict(X_train))  # Threshold at 0.5 for binary classification
y_pred_valid = np.round(gbm.predict(X_valid))  # Threshold at 0.5 for binary classification

# Calculate accuracy
train_accuracy = np.mean(y_pred_train == y_train)
valid_accuracy = np.mean(y_pred_valid == y_valid)

print(f"Training Accuracy: {train_accuracy:.3f}")
print(f"Validation Accuracy: {valid_accuracy:.3f}")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.70956
[100]	valid_0's auc: 0.717443
Early stopping, best iteration is:
[101]	valid_0's auc: 0.718126
Training Accuracy: 0.964
Validation Accuracy: 0.970


Evaluation with AUC and then comparison with the stability metric is shown below.

In [ ]:
# for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
#     y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
#     base["score"] = y_pred

# print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}')
# print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}')
# print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')


# def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
#     gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
#         .sort_values("WEEK_NUM")\
#         .groupby("WEEK_NUM")[["target", "score"]]\
#         .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()

#     x = np.arange(len(gini_in_time))
#     y = gini_in_time
#     a, b = np.polyfit(x, y, 1)
#     y_hat = a*x + b
#     residuals = y - y_hat
#     res_std = np.std(residuals)
#     avg_gini = np.mean(gini_in_time)
#     return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

# stability_score_train = gini_stability(base_train)
# stability_score_valid = gini_stability(base_valid)
# stability_score_test = gini_stability(base_test)

# print('\n')
# print(f'The stability score on the train set is: {stability_score_train}')
# print(f'The stability score on the valid set is: {stability_score_valid}')
# print(f'The stability score on the test set is: {stability_score_test}')

import lightgbm as lgb
import numpy as np
import pickle

# Define datasets for LightGBM
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

# Set parameters for the model
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 4,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

# Train the model
gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_valid],
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

# Generate raw predictions (these are probabilities, not binary labels)
y_pred_train = gbm.predict(X_train)  # Raw probabilities
y_pred_valid = gbm.predict(X_valid)  # Raw probabilities

# Save the raw predictions to pickle files
with open('y_pred_train.pkl', 'wb') as f:
    pickle.dump(y_pred_train, f)

with open('y_pred_valid.pkl', 'wb') as f:
    pickle.dump(y_pred_valid, f)

# Convert raw probabilities to binary predictions using 0.5 threshold
y_pred_train_binary = np.round(y_pred_train)  # Threshold at 0.5 for binary classification
y_pred_valid_binary = np.round(y_pred_valid)  # Threshold at 0.5 for binary classification

# Calculate accuracy for binary predictions
train_accuracy = np.mean(y_pred_train_binary == y_train)
valid_accuracy = np.mean(y_pred_valid_binary == y_valid)

print(f"Training Accuracy: {train_accuracy:.3f}")
print(f"Validation Accuracy: {valid_accuracy:.3f}")

Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.70956
[100]	valid_0's auc: 0.717443
Early stopping, best iteration is:
[101]	valid_0's auc: 0.718126
Training Accuracy: 0.964
Validation Accuracy: 0.970


# Submission

Scoring the submission dataset is below, we need to take care of new categories. Then we save the score as a last step.

In [ ]:
# # 데이터 타입 일치 여부 확인
# dtype_match = X_train.dtypes == X_submission.dtypes

# if dtype_match.all():
#     print("데이터 타입이 일치합니다.")
# else:
#     print("데이터 타입이 일치하지 않습니다.")

#     # 불일치한 열과 데이터 타입 출력
#     mismatched_cols = X_train.columns[~dtype_match].tolist()
#     for col in mismatched_cols:
#         print(f"데이터 타입 불일치 - 열: {col}, 훈련 데이터셋: {X_train[col].dtype}, 제출 데이터셋: {X_submission[col].dtype}")

In [ ]:
# X_submission = data_submission[cols_pred].to_pandas()
# X_submission = convert_strings(X_submission)
# categorical_cols = X_train.select_dtypes(include=['category']).columns

# for col in categorical_cols:
#     train_categories = set(X_train[col].cat.categories)
#     submission_categories = set(X_submission[col].cat.categories)
#     new_categories = submission_categories - train_categories
#     X_submission.loc[X_submission[col].isin(new_categories), col] = "Unknown"
#     new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
#     X_train[col] = X_train[col].astype(new_dtype)
#     X_submission[col] = X_submission[col].astype(new_dtype)

# y_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration)


# submission = pd.DataFrame({
#     "case_id": data_submission["case_id"].to_numpy(),
#     "score": y_submission_pred
# }).set_index('case_id')
# submission.to_csv("./submission.csv")

In [ ]:
# # 특성 중요도 가져오기
# importances = gbm.feature_importance()

# # 중요도를 데이터프레임으로 변환
# feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importances})

# # 중요도에 따라 특성 정렬
# feature_importances = feature_importances.sort_values('importance', ascending=False)

# # 중요도가 높은 특성 출력
# print(feature_importances)

In [ ]:
# # 중요도 막대 그래프 그리기
# import matplotlib.pyplot as plt
# plt.figure(figsize=(15, 15))
# plt.title("Feature importances")
# plt.barh(feature_importances['feature'], feature_importances['importance'], color='b', align='center')
# plt.xlabel("Importance")
# plt.ylabel("Features")
# plt.gca().invert_yaxis()
# plt.show()

In [ ]:
# import shap

# explainer = shap.Explainer(gbm)
# shap_values = explainer.shap_values(X_test)

# shap.summary_plot(shap_values, X_test, plot_type='bar')

In [ ]:
# shap.summary_plot(shap_values, X_train)

In [ ]:
# explanation_object = explainer(X_test)

In [ ]:
# shap.plots.waterfall(explanation_object[0])